###  Things to install for the process

In [1]:
#Things needed to download the video
!pip install --upgrade pytubefix
!pip show pytube

# Library for audio to text
!pip install openai-whisper

# Library for scenedetection
!pip install scenedetect
!pip install pytubefix

# Library for extracting the metadata
!pip install tinytag

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 11.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=eb3843dea0f0073bdfac7d40e741ebfb8e98e70f60039b8917e36e2ba18db1b5
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.9/130.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.3.0
    Uninstalling click-8.3.0:
      Successfully uninstalled click-8.3.0


### Library for the video Application

In [2]:
# Library to import for the video download
from pytubefix import YouTube
from pytubefix.cli import on_progress

# Library to import for the ContentDetection
from scenedetect import VideoManager, SceneManager
from scenedetect.detectors import ContentDetector
from moviepy.editor import VideoFileClip

# Library for extracting the metadata
from tinytag import TinyTag

# Library for extracting audio
import moviepy.editor as mp

#Library needed for the scenedetection
import scenedetect
from scenedetect import open_video
from scenedetect import detect, ContentDetector, split_video_ffmpeg

#Library for concating the video
from moviepy.editor import concatenate_videoclips

/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
  if event.key is 'enter':



#### Downloading Video is finished

In [3]:
def download_youtube_video(filename="Sample.mp4"):
    try:
        #Asking for the url
        url=str(input("Enter the url of the video"))

        # Create a YouTube object
        yt = YouTube(url)

        # Get the Video
        stream = yt.streams.filter(file_extension="mp4", progressive=True).first()

        # Download the video with a specific filename (if provided)
        if filename:
            downloaded_file = stream.download(filename=filename)
        else:
            downloaded_file = stream.download()

        # Print the name of the downloaded file
        print(f"Video downloaded successfully: {downloaded_file}")

    except Exception as e:
        print(f"An error occurred: {e}")

    return downloaded_file

In [ ]:
x=download_youtube_video()

Enter the url of the videohttps://youtu.be/eNvUS-6PTbs?si=Rl5a-j1y2ugo6A1n
Video downloaded successfully: /content/Sample.mp4


### Extracted subtitle from the yt.video

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
from urllib.parse import urlparse, parse_qs
import pandas as pd

In [ ]:
def extract_video_id(url:str):
  parsed=urlparse(url)
  if parsed.hostname in {"www.youtube.com","youtube.com"}:
    qs=parse_qs(parsed.query)
    if "v" in qs:
      return qs["v"][0]
  return parsed.path.lstrip("/")

In [ ]:
def extract_sub(url:str):
  """
  url : Enter the url of the video you need to extract


  Enter the Url and the function will try to get the
  subtitle of the video and the timestamp of the subtitle,
  Create a dataframe out of it"""

  video_id=extract_video_id(url)
  ytt_api=YouTubeTranscriptApi()
  subtitle=ytt_api.fetch(video_id)
  data =[{"text": s.text, "start": s.start, "duration": s.duration} for s in subtitle.snippets]
  dataframe=pd.DataFrame(data)
  return dataframe

In [ ]:
data=extract_sub(url="https://www.youtube.com/watch?v=U_v2cnTav7Y")

In [ ]:
data

,text,start,duration
0,This is the OnePlus 15. And I'll be,0.08,4.000
1,"honest with you guys, OnePlus might fall",1.92,6.919
2,short against the competition this time.,4.08,4.759
3,[Music],18.98,3.289
4,"So, it's coming with this OnePlus 13S",23.60,4.160
...,...,...,...
58,days. Thanks so much for watching. Let,175.20,3.280
59,me know your thoughts in the comment,177.04,3.760
60,section below and I'll meet you guys in,178.48,4.560
61,"the next one. Like, share, subscribe,",180.80,5.719


### Extracting the audio with the timestamp

In [ ]:
!pip install openai-whisper moviepy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 13.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=52b3a3a22358024e2fd2a744184b7673f5a547e27933ba3e2aaa3ca27c270acd
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [ ]:
x=download_youtube_video()

Enter the url of the videohttps://www.youtube.com/watch?v=U_v2cnTav7Y\
Video downloaded successfully: /content/Sample.mp4


In [ ]:
from moviepy.editor import VideoFileClip

In [ ]:
video = VideoFileClip(x)
video.audio.write_audiofile("audio.wav")

MoviePy - Writing audio in audio.wav


MoviePy - Done.


In [ ]:
import whisper

model = whisper.load_model("base")
result = model.transcribe("audio.wav", verbose=True)
print(result["segments"])

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 122MiB/s]
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: English
[00:00.000 --> 00:02.720]  This C1 Plus 15, and I'll be honest with you guys,
[00:02.720 --> 00:06.720]  OnePlus might fall short against the competition this time.
[00:23.440 --> 00:27.120]  So it's coming with this OnePlus 30-ness kind of design language.
[00:27.120 --> 00:31.440]  Similar Squarish module containing triple cameras and a LED flash,
[00:31.440 --> 00:35.920]  but with a larger form factor at 6.7 inch or 6.8 inch I guess.
[00:40.160 --> 00:44.320]  OnePlus is using a boxy design with nano ceramic metal frame.
[00:44.320 --> 00:48.880]  Yeah, I kind of like this minimal finish, but when you compare it to a latest pixel,
[00:48.880 --> 00:55.040]  Samsung Ultra iPhone 17 or Xiaomi 17 lineup, it doesn't stand out cause of no
[00:55.040 --> 00:57.920]  less innovation in terms of the design.
[01:00.160 --> 01:03.040]  So a generic mid-range kind of design

In [ ]:
with open("subtitles.srt", "w") as f:
    for i, seg in enumerate(result["segments"], 1):
        f.write(f"{i}\n")
        f.write(f"{seg['start']:.3f} --> {seg['end']:.3f}\n")
        f.write(f"{seg['text']}\n\n")


In [ ]:
import pandas as pd

# Assuming result["segments"] is a list of dicts with 'start', 'end', and 'text'
segments = result["segments"]

# Convert to DataFrame
df = pd.DataFrame(segments)

# Display the table
print(df[['start', 'end', 'text']])


     start     end                                               text
0     0.00    2.72   This C1 Plus 15, and I'll be honest with you ...
1     2.72    6.72   OnePlus might fall short against the competit...
2    23.44   27.12   So it's coming with this OnePlus 30-ness kind...
3    27.12   31.44   Similar Squarish module containing triple cam...
4    31.44   35.92   but with a larger form factor at 6.7 inch or ...
5    40.16   44.32   OnePlus is using a boxy design with nano cera...
6    44.32   48.88   Yeah, I kind of like this minimal finish, but...
7    48.88   55.04   Samsung Ultra iPhone 17 or Xiaomi 17 lineup, ...
8    55.04   57.92            less innovation in terms of the design.
9    60.16   63.04       So a generic mid-range kind of design it is,
10   63.92   65.84                              not anything special.
11   68.48   71.44     The rumored specifications however are decent.
12   71.44   75.28        OnePlus is using a 165Hz high refresh rate,
13   75.28   80.40  

In [ ]:
x=download_youtube_video()

Enter the url of the videohttps://youtu.be/GJCj4lfraDc?si=Ca8hH7yQhpK1nyB3
Video downloaded successfully: /content/Sample.mp4


In [ ]:
extract_sub("https://youtu.be/GJCj4lfraDc?si=Ca8hH7yQhpK1nyB3")

TranscriptsDisabled: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=GJCj4lfraDc! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!

### Combine both to get any of one

In [ ]:
def download_subtitle_or_transcribe(url,download_path):
  """ Try to download the subtitle else audio Transcription"""
  try:
    video_id=extract_video_id(url)
    ytt_api=YouTubeTranscriptApi()
    subtitle=ytt_api.fetch(video_id)
    data =[{"text": s.text, "start": s.start, "duration": s.duration} for s in subtitle.snippets]
    dataframe=pd.DataFrame(data)
    return dataframe
  except Exception as e:
    video = VideoFileClip(download_path)
    video.audio.write_audiofile("audio.wav")
    model = whisper.load_model("base")
    result = model.transcribe("audio.wav", verbose=False)
    #print(result["segments"])
    segments = result["segments"]
    # Convert to DataFrame
    df = pd.DataFrame(segments)
    selected_df = df[['start', 'end', 'text']]
    # Display the table
    return selected_df


In [ ]:
download_subtitle_or_transcribe(url="https://youtu.be/eNvUS-6PTbs?si=Rl5a-j1y2ugo6A1n")

MoviePy - Writing audio in audio.wav


MoviePy - Done.


  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



Detected language: English


100%|█████████▉| 19700/19704 [00:36<00:00, 545.89frames/s]


,start,end,text
0,0.0,18.0,Music
1,18.0,22.0,Oh I can not explain every time it's the same
2,22.0,26.0,More I feel they're free to take my heart
3,26.0,31.0,"I've been lonely too long, oh I can't be so s..."
4,31.0,34.0,Take the chance for a man to take my heart
5,34.0,42.0,"I need you soon, there's no time I'll ever go"
6,42.0,47.0,"Sherry Sherry Lady, go into a motion"
7,47.0,51.0,"Love is where you find it, in the sun to your..."
8,51.0,55.0,"I'm Sherry Sherry Lady, living in the motion"
9,55.0,59.0,"It's always like the first time, let me take ..."


### calculating the entire time for running!!!

In [ ]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.2/86.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.4/718.4 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.4/

In [ ]:
#Things needed to download the video
!pip install --upgrade pytubefix
!pip show pytube
# Library for audio to text
!pip install openai-whisper
# Library for scenedetection
!pip install scenedetect
# Library for extracting the metadata
!pip install tinytag

In [ ]:
import mlflow
import time

In [ ]:
# Library to import for the video download
from pytubefix import YouTube
from pytubefix.cli import on_progress

# Library to import for the ContentDetection
from scenedetect import VideoManager, SceneManager
from scenedetect.detectors import ContentDetector
from moviepy.editor import VideoFileClip

# Library for extracting the metadata
from tinytag import TinyTag

# Library for extracting audio
import moviepy.editor as mp

#Library for concating the video
from moviepy.editor import concatenate_videoclips

import whisper
import pandas as pd
import mlflow
import time

from youtube_transcript_api import YouTubeTranscriptApi
from urllib.parse import urlparse, parse_qs
import pandas as pd

In [ ]:
def download_youtube_video(filename="Sample.mp4"):
    try:
        #Asking for the url
        url="https://youtu.be/eNvUS-6PTbs?si=Rl5a-j1y2ugo6A1n"

        # Create a YouTube object
        yt = YouTube(url)

        # Get the Video
        stream = yt.streams.filter(file_extension="mp4", progressive=True).first()

        # Download the video with a specific filename (if provided)
        if filename:
            downloaded_file = stream.download(filename=filename)
        else:
            downloaded_file = stream.download()

        # Print the name of the downloaded file
        print(f"Video downloaded successfully: {downloaded_file}")

    except Exception as e:
        print(f"An error occurred: {e}")

    return url, downloaded_file

In [ ]:
def extract_video_id(url:str):
  parsed=urlparse(url)
  if parsed.hostname in {"www.youtube.com","youtube.com"}:
    qs=parse_qs(parsed.query)
    if "v" in qs:
      return qs["v"][0]
  return parsed.path.lstrip("/")

In [ ]:
def extract_sub(url:str):
  """
  url : Enter the url of the video you need to extract


  Enter the Url and the function will try to get the
  subtitle of the video and the timestamp of the subtitle,
  Create a dataframe out of it"""

  video_id=extract_video_id(url)
  ytt_api=YouTubeTranscriptApi()
  subtitle=ytt_api.fetch(video_id)
  data =[{"text": s.text, "start": s.start, "duration": s.duration} for s in subtitle.snippets]
  dataframe=pd.DataFrame(data)
  return dataframe

In [ ]:
import whisper
from pydub import AudioSegment

In [ ]:
def download_subtitle_or_transcribe(url,download_path):
  """ Try to download the subtitle else audio Transcription"""
  try:
    video_id=extract_video_id(url)
    ytt_api=YouTubeTranscriptApi()
    subtitle=ytt_api.fetch(video_id)
    data =[{"text": s.text, "start": s.start, "duration": s.duration} for s in subtitle.snippets]
    dataframe=pd.DataFrame(data)
    return dataframe
  except Exception as e:
    video = VideoFileClip(download_path)
    video.audio.write_audiofile("audio.wav")

    # Faster inference
    model = WhisperModel("base", device="cpu", compute_type="int8")
    segments, info = model.transcribe("audio.wav", beam_size=5)
    segments = list(segments)


    df = pd.DataFrame(segments)
    selected_df = df[['start', 'end', 'text']]
    return print(selected_df)

In [ ]:
x,y=download_youtube_video()
download_subtitle_or_transcribe(url=x, download_path=y)

Video downloaded successfully: /content/Sample.mp4
MoviePy - Writing audio in audio.wav


MoviePy - Done.


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



KeyboardInterrupt: 

In [ ]:
with mlflow.start_run(run_name="Using-faster-whisper",description="audio trans"):
  start_time=time.time()
  x,y=download_youtube_video()
  download_subtitle_or_transcribe(url=x, download_path=y)
  end_time=time.time()
  runtime=end_time - start_time
  mlflow.log_metric("runtime_seconds", runtime)

  print(f"\nTotal Runtime: {runtime:.4f} seconds")

Video downloaded successfully: /content/Sample.mp4
MoviePy - Writing audio in audio.wav


MoviePy - Done.
    start    end                                               text
0     0.0   18.0                                              Music
1    18.0   22.0      Oh, I cannot explain every time it's the same
2    22.0   26.0          More I feel they're free to take my heart
3    26.0   31.0   I've been lonely too long, oh, I can't be so ...
4    31.0   37.0   To change for a man to take my heart, I need ...
5    37.0   42.0                      There's no time, I'll ever go
6    42.0   47.0              Sherry, Sherry lady, go into a motion
7    47.0   51.0   Love is where you find it, in the sun to your...
8    51.0   55.0      I'm Sherry, Sherry lady, living in the motion
9    55.0   59.0   It's always like the first time, let me take ...
10   59.0   64.0      I'm Sherry, Sherry lady, one person to follow
11   64.0   70.0   Take my heart, listen to your Sherry, Sherry ...
12   70.0   74.0   It's always like the first time, let me take ...
13   74.0   78.0      I'm Sherry

### Model runtime ( audio trans )

- Baseline model :- 48.7809 seconds
- UsingFFmpeg-Whisper1 :- 51.4701 seconds
- Direct Transcription :- 44.3696 seconds
- PyDub Whisper :- 45.2708 seconds
- Using faster-whisper :- 92.6830 seconds


In [ ]:
#!rm -rf /content/mlruns/0/47fe4283cdc744e9965d1b73cc852aea

### Created the below function by chapgpt

In [ ]:
import os
import json
import yaml
import pandas as pd

def parse_mlruns(mlruns_path="/content/mlruns"):
    """
    Parses the mlruns folder to create a table for analysis.

    Args:
        mlruns_path (str): Path to the mlruns folder (default is "/content/mlruns").

    Returns:
        pd.DataFrame: A DataFrame containing run details (run_id, experiment_id, metrics, params, etc.).
    """
    runs_data = []

    # Iterate through each experiment folder in mlruns
    for experiment_id in os.listdir(mlruns_path):
        experiment_path = os.path.join(mlruns_path, experiment_id)
        if not os.path.isdir(experiment_path):
            continue  # Skip non-directory files

        # Iterate through each run folder inside the experiment
        for run_id in os.listdir(experiment_path):
            run_path = os.path.join(experiment_path, run_id)
            if not os.path.isdir(run_path):
                continue

            run_data = {"experiment_id": experiment_id, "run_id": run_id}

            # ---- Load meta.yaml ----
            meta_file = os.path.join(run_path, "meta.yaml")
            if os.path.exists(meta_file):
                with open(meta_file, "r") as f:
                    meta = yaml.safe_load(f)
                    if isinstance(meta, dict):
                        run_data.update(meta)

            # ---- Load metrics ----
            metrics_dir = os.path.join(run_path, "metrics")
            if os.path.exists(metrics_dir):
                for metric_file in os.listdir(metrics_dir):
                    metric_path = os.path.join(metrics_dir, metric_file)
                    if os.path.isfile(metric_path):
                        with open(metric_path, "r") as f:
                            lines = f.readlines()
                            # Each metric file can contain multiple lines like: step, value, timestamp
                            last_line = lines[-1].strip().split()
                            if len(last_line) >= 2:
                                value = float(last_line[1])
                                run_data[metric_file] = value

            # ---- Load params ----
            params_dir = os.path.join(run_path, "params")
            if os.path.exists(params_dir):
                for param_file in os.listdir(params_dir):
                    param_path = os.path.join(params_dir, param_file)
                    if os.path.isfile(param_path):
                        with open(param_path, "r") as f:
                            run_data[param_file] = f.read().strip()

            # ---- Convert runtime from ms → seconds ----
            # Some MLflow meta.yaml files have "end_time" and "start_time" in milliseconds
            if "start_time" in run_data and "end_time" in run_data:
                try:
                    start = int(run_data["start_time"])
                    end = int(run_data["end_time"])
                    run_data["runtime_seconds"] = round((end - start) / 1000, 3)
                except (ValueError, TypeError):
                    run_data["runtime_seconds"] = None

            runs_data.append(run_data)

    # Convert to DataFrame
    if not runs_data:
        print("No runs found in:", mlruns_path)
        return pd.DataFrame()

    return pd.DataFrame(runs_data)


In [ ]:
parse_mlruns()

,experiment_id,run_id,artifact_uri,end_time,entry_point_name,lifecycle_stage,run_name,source_name,source_type,source_version,start_time,status,tags,user_id,runtime_seconds
0,0,d800b7be6b1f452a91a148aa24791355,file:///content/mlruns/0/d800b7be6b1f452a91a14...,1760033890150,,active,Baseline-model_run1,,4,,1760033841519,3,[],root,48.631
1,0,73bb2b8f2771466abe54459d26ecbc33,file:///content/mlruns/0/73bb2b8f2771466abe544...,1760034750417,,active,PyDub-Whisper,,4,,1760034705123,3,[],root,45.294
2,0,33514a226b13465ca6a0f8dd11fdb7b7,file:///content/mlruns/0/33514a226b13465ca6a0f...,1760035583964,,active,Using-faster-whisper,,4,,1760035491268,3,[],root,92.696
3,0,089cee2123d142f998815b93c0d2fbe9,file:///content/mlruns/0/089cee2123d142f998815...,1760034580533,,active,Direct-Transcription,,4,,1760034536151,3,[],root,44.382
4,0,68674e4a677e4243a1018d4029a26603,file:///content/mlruns/0/68674e4a677e4243a1018...,1760034396938,,active,UsingFFmpeg-Whisper1,,4,,1760034345449,3,[],root,51.489


In [ ]:
data=parse_mlruns()

In [ ]:
df = data[['run_name', 'start_time','end_time', 'runtime_seconds']]

In [ ]:
# The Compartion of different library for extracting the subtitle
df

,run_name,start_time,end_time,runtime_seconds
0,Baseline-model_run1,1760033841519,1760033890150,48.631
1,PyDub-Whisper,1760034705123,1760034750417,45.294
2,Using-faster-whisper,1760035491268,1760035583964,92.696
3,Direct-Transcription,1760034536151,1760034580533,44.382
4,UsingFFmpeg-Whisper1,1760034345449,1760034396938,51.489


In [ ]:
## downloading the video
import shutil
from google.colab import files

# Specify the folder you want to download
folder_to_download = '/content/mlruns'

# Compress the folder into a .zip file
shutil.make_archive(folder_to_download, 'zip', folder_to_download)

# Download the .zip file
files.download(f'{folder_to_download}.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### SceneDetection

In [1]:
!pip install scenedetect
!pip install pytubefix

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.9/130.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.3.0
    Uninstalling click-8.3.0:
      Successfully uninstalled click-8.3.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 MB 14.6 MB/s eta 0:00:00


In [69]:
# Library to import for the video download
from pytubefix import YouTube
from pytubefix.cli import on_progress

#Library needed for the scenedetection
import scenedetect
from scenedetect import open_video
from scenedetect import detect, ContentDetector, split_video_ffmpeg


In [70]:
def download_youtube_video(filename="Sample.mp4"):
    try:
        #Asking for the url
        url=str(input("Enter the url of the video"))

        # Create a YouTube object
        yt = YouTube(url)

        # Get the Video
        stream = yt.streams.filter(file_extension="mp4", progressive=True).first()

        # Download the video with a specific filename (if provided)
        if filename:
            downloaded_file = stream.download(filename=filename)
        else:
            downloaded_file = stream.download()

        # Print the name of the downloaded file
        print(f"Video downloaded successfully: {downloaded_file}")

    except Exception as e:
        print(f"An error occurred: {e}")

    return downloaded_file

In [71]:
x=download_youtube_video()

Enter the url of the videohttps://youtu.be/eNvUS-6PTbs?si=3t22XDOqSqbuq5mJ
Video downloaded successfully: /content/Sample.mp4


In [72]:
video=open_video(x)
while True:
  frame= video.read()
  if frame is False:
    break
print("Read %d frames" % video.frame_number)

Read 4926 frames


In [73]:
scenes=detect(x, ContentDetector())

INFO:pyscenedetect:Detecting scenes...


In [74]:
scenes=detect(x, ContentDetector())
data=[]
for (scene_start, scene_end) in scenes:
  data.append({"Scence_start":scene_start, "Scence_end":scene_end})
  print(f"{scene_start}-{scene_end}")

INFO:pyscenedetect:Detecting scenes...


00:00:00.000-00:00:01.840
00:00:01.840-00:00:03.120
00:00:03.120-00:00:06.120
00:00:06.120-00:00:08.200
00:00:08.200-00:00:15.360
00:00:15.360-00:00:16.680
00:00:16.680-00:00:18.200
00:00:18.200-00:00:30.360
00:00:30.360-00:00:42.840
00:00:42.840-00:00:44.840
00:00:44.840-00:00:46.840
00:00:46.840-00:00:48.920
00:00:48.920-00:00:49.920
00:00:49.920-00:00:50.960
00:00:50.960-00:00:58.560
00:00:58.560-00:00:59.840
00:00:59.840-00:01:01.640
00:01:01.640-00:01:05.760
00:01:05.760-00:01:07.760
00:01:07.760-00:01:10.000
00:01:10.000-00:01:12.000
00:01:12.000-00:01:16.120
00:01:16.120-00:01:17.040
00:01:17.040-00:01:18.680
00:01:18.680-00:01:28.120
00:01:28.120-00:01:35.840
00:01:35.840-00:01:43.440
00:01:43.440-00:01:44.680
00:01:44.680-00:01:51.640
00:01:51.640-00:01:59.400
00:01:59.400-00:02:34.360
00:02:34.360-00:02:40.640
00:02:40.640-00:02:43.360
00:02:43.360-00:02:48.120
00:02:48.120-00:02:49.120
00:02:49.120-00:02:56.120
00:02:56.120-00:02:56.840
00:02:56.840-00:02:59.760
00:02:59.760

In [78]:
import pandas as pd
data1=pd.DataFrame(data)
data[1]

{'Scence_start': 00:00:01.840 [frame=46, fps=25.000],
 'Scence_end': 00:00:03.120 [frame=78, fps=25.000]}

In [38]:
scenedetect.frame_timecode.FrameTimecode(timecode=3559, fps=46)

00:01:17.370 [frame=3559, fps=46.000]

### Merge Video

In [49]:
from moviepy.editor import VideoFileClip, concatenate_videoclips
from scenedetect import FrameTimecode

In [64]:
clip=VideoFileClip(x)
fps=25.0

In [79]:
 scene=data[2]

In [81]:
for i in split_indices:
  print(data[i])

{'Scence_start': 00:00:01.840 [frame=46, fps=25.000], 'Scence_end': 00:00:03.120 [frame=78, fps=25.000]}
{'Scence_start': 00:00:16.680 [frame=417, fps=25.000], 'Scence_end': 00:00:18.200 [frame=455, fps=25.000]}
{'Scence_start': 00:00:18.200 [frame=455, fps=25.000], 'Scence_end': 00:00:30.360 [frame=759, fps=25.000]}
{'Scence_start': 00:00:46.840 [frame=1171, fps=25.000], 'Scence_end': 00:00:48.920 [frame=1223, fps=25.000]}


In [82]:
split_indices=[1,6, 7, 11]
split_clips=[]
clip=VideoFileClip(x)

for i in split_indices:
  scene=data[i]
  start_tc=scene["Scence_start"]
  end_tc=scene["Scence_end"]

  start_sec=start_tc.get_seconds()
  end_sec=end_tc.get_seconds()
  #mid_sec = start_sec + (end_sec - start_sec) / 2

  split_clips.append(clip.subclip(start_sec, end_sec))
  """    split_clips.append(clip.subclip(start_sec, mid_sec))
    split_clips.append(clip.subclip(mid_sec, end_sec))"""


final_clip=concatenate_videoclips(split_clips)

for i in split_clips:
  final_clip=concatenate_videoclips([i])

In [48]:
split_clips

In [49]:
final_clip = concatenate_videoclips(split_clips)

# Export the result
final_clip.write_videofile("merged_video.mp4")

Moviepy - Building video merged_video.mp4.
MoviePy - Writing audio in merged_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video merged_video.mp4



Moviepy - Done !
Moviepy - video ready merged_video.mp4


In [10]:
import pandas as pd

In [23]:
x=download_youtube_video()

Enter the url of the videohttps://youtu.be/eNvUS-6PTbs?si=Gwyn54cNEj8ytA3m
Video downloaded successfully: /content/Sample.mp4


In [114]:
def scence_detected(path:str)-> str:
  scenes=detect(path, ContentDetector())
  data=[]
  for (scene_start, scene_end) in scenes:
    data.append({"Scene_start":scene_start, "Scene_end":scene_end})
  show_data=pd.DataFrame(data)
  return data, show_data

In [115]:
data, show_data=scence_detected(x)

INFO:pyscenedetect:Detecting scenes...


In [117]:
show_data.tail()

,Scene_start,Scene_end
35,00:02:49.120,00:02:56.120
36,00:02:56.120,00:02:56.840
37,00:02:56.840,00:02:59.760
38,00:02:59.760,00:03:16.400
39,00:03:16.400,00:03:17.040


In [112]:
data[0]

{'Scene_start': 00:00:00.000 [frame=0, fps=25.000],
 'Scene_end': 00:00:01.840 [frame=46, fps=25.000]}

In [122]:
from typing import Dict, List
def merge_video(data=List[Dict],path=str,clip=list):
  """
  data: The List of start and end point of scenedetection
  path: Path of the video
  clip[List]: The list of scene need to be merge
  """

  clip_path=VideoFileClip(path)
  split_clips=[]

  for arg in clip:
    scene=data[arg]
    start_tc=scene["Scene_start"]
    end_tc=scene["Scene_end"]

    start_sec=start_tc.get_seconds()
    end_sec=end_tc.get_seconds()

    split_clips.append(clip_path.subclip(start_sec, end_sec))

  final_clip=concatenate_videoclips(split_clips)
  final_clip.write_videofile("merge.mp4")

In [120]:
list1=[2, 10, 16, 28]
result = [x - 1 for x in list1]


In [46]:
x

'/content/Sample.mp4'

In [121]:
merge_video(data, x, result )

Moviepy - Building video merge.mp4.
MoviePy - Writing audio in mergeTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video merge.mp4



Moviepy - Done !
Moviepy - video ready merge.mp4


### Looking the video

In [16]:
from IPython.display import Video

In [16]:
from IPython.display import HTML

In [19]:
Video('/content/Sample.mp4')

### Find the main part of the video

DEEP DIVE BRUH!!!